# 异常错误处理
- 重试
- 修复

# 重试 (RetryOutputParser)

In [1]:
# --- 1. 引入依赖包 ---
import os
from langchain.output_parsers import RetryOutputParser
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# --- 2. 定义我们期望的数据结构 ---
# 这里我们期望LLM返回一个包含 "action" 和 "action_input" 的指令
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")

# --- 3. 实例化一个标准的Pydantic解析器 ---
parser = PydanticOutputParser(pydantic_object=Action)

# --- 4. 定义提示词模板 ---
# 这个模板指导LLM如何根据用户问题提供Action
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# --- 5. 模拟一个失败的场景 ---

# 假设用户的原始问题是 "北京今天天气如何？"
# 我们先用模板生成完整的提示内容
prompt_value = prompt.format_prompt(query="北京今天天气如何？")

# 假设LLM返回了一个格式错误的回答
# 这个回答只包含了 action，缺少了 action_input，不符合 Action 模型的要求
bad_response = '{"action": "search"}'

# 我们尝试用标准解析器来解析这个错误回答，它会失败并抛出异常
print("--- 演示标准解析器的失败 ---")
try:
    parser.parse(bad_response)
except OutputParserException as e:
    print("解析失败，错误信息:")
    print(e) # 打印出的错误会提示 action_input 字段缺失

# --- 6. 使用 RetryOutputParser 实现错误重试 ---

# 首先，定义一个用于重试和修正错误的LLM
import os
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

# 定义一个重试解析器
# 它需要一个LLM来进行修正，以及一个原始的解析器来指明修正目标
retry_parser = RetryOutputParser.from_llm(llm=llm, parser=parser)

# 传入错误的回答以及原始的提示值，让它来尝试修正并解析
print("\n--- 使用 RetryOutputParser 自动修正 ---")
parsed_response = retry_parser.parse_with_prompt(bad_response, prompt_value)

print("修正并解析成功！结果是：")
print(parsed_response)
print("\n现在可以像对象一样访问字段了：")
print("Action:", parsed_response.action)
print("Action Input:", parsed_response.action_input)

C:\Users\jz292\.conda\envs\langchain-learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- 演示标准解析器的失败 ---
解析失败，错误信息:
Failed to parse Action from completion {"action": "search"}. Got: 1 validation error for Action
action_input
  Field required [type=missing, input_value={'action': 'search'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

--- 使用 RetryOutputParser 自动修正 ---
修正并解析成功！结果是：
action='weather_query' action_input='北京'

现在可以像对象一样访问字段了：
Action: weather_query
Action Input: 北京


# 错误修复

In [2]:
# --- 1. 引入依赖包 ---
import os
from langchain.output_parsers import OutputFixingParser
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# --- 2. 定义一个演员模型，有两个字段 ---
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

# --- 3. 实例化一个标准的Pydantic解析器 ---
parser = PydanticOutputParser(pydantic_object=Actor)

# --- 4. 模拟一个失败的场景 ---

# 假设LLM返回了一个格式错误的字符串
# 这里的 'film_names' 后面多了一个不匹配的 ']'，导致它不是一个有效的JSON
misformatted = "{'name': 'Tom Hanks', 'film_names': ['Forrest Gump']]}"

# 尝试用标准解析器解析，会失败并抛出错误
print("--- 演示标准解析器的失败 ---")
try:
    parser.parse(misformatted)
except OutputParserException as e:
    print("解析失败，错误信息:")
    print(e)

# --- 5. 使用 OutputFixingParser 进行错误修复 ---

# 定义一个可以修复错误的LLM
# 注意：你需要设置你的OpenAI API密钥
from langchain_deepseek import ChatDeepSeek
llm = ChatDeepSeek(
    temperature=0,  # temperature=0表示输出更加确定性，不会随机性太强
    model_name="deepseek-chat",  # 模型名称
    api_key=os.getenv("DEEPSEEK_API_KEY")  # 从环境变量中获取API密钥
)

# 定义一个输出修复解析器 (OutputFixingParser)
# 它需要一个LLM来进行修复，以及一个原始的解析器来指明修复目标
new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# 传入错误的信息，让它来尝试修复并解析
print("\n--- 使用 OutputFixingParser 自动修复 ---")
result = new_parser.parse(misformatted)

print("修复并解析成功！结果是：")
print(result)

print("\n现在可以像对象一样访问字段了：")
print("Name:", result.name)
print("Film Names:", result.film_names)

--- 演示标准解析器的失败 ---
解析失败，错误信息:
Failed to parse Actor from completion null. Got: 1 validation error for Actor
  Input should be a valid dictionary or instance of Actor [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

--- 使用 OutputFixingParser 自动修复 ---
修复并解析成功！结果是：
name='Tom Hanks' film_names=['Forrest Gump', 'Saving Private Ryan', 'Cast Away']

现在可以像对象一样访问字段了：
Name: Tom Hanks
Film Names: ['Forrest Gump', 'Saving Private Ryan', 'Cast Away']
